In [149]:
# Specifying all the imports
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts.prompt import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from dotenv import load_dotenv
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

import os

In [173]:
# Loading dotenv and specifying tools
load_dotenv()

@tool
def search_tool(query : str):
    """ 
    Takes a user query and performs an online search to provide
    an answer to the query
    """
    
    search = DuckDuckGoSearchRun()
    answer = search.run(query)
    return answer

@tool
def create_embeddings(query : str):
    """
    Uses the content from the file path to create embeddings on the inputted data. Using the query
    provided, perform a similarity search on the query and return the output. If you can not provide a response,
    say I don't know rather than providing an incorrect response.
    """

    #file_path = r'C:\GenAI\Final Project\GenAIGroupProject\apple.txt'
    file_path = r'./apple.txt'
    file_path = file_path.strip()
    loader = TextLoader(file_path, encoding= 'utf-8')
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunk = text_splitter.split_documents(documents=document)
    embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))
    vectorstore = FAISS.from_documents(chunk, embeddings)

    docs = vectorstore.similarity_search(query)
    return docs[0].page_content


@tool
def query_database(query : str):
    """
    This function queries the SQL database based on the query that the user provides
    to provide insight into the current client stock portfolio
    """

    

    # Initialize the JSON output key parser


    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)
    db = SQLDatabase.from_uri('sqlite:///portfolio_allocations.db')
    agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
    result = agent_executor.invoke(query)


    response_schemas = [
        ResponseSchema(name="client", description="get client as a key for a dictionary"),
        ResponseSchema(
            name="stock allocations",
            description="returns a list of dictionaries of each stock with their ticker as a key and percentage allocations as a value for each client",
        ),
    ]

    
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    


    format_instructions = output_parser.get_format_instructions()
    
    prompt = PromptTemplate(
    template="create a new key, value pair for every new client mentioned in the question.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

    
    
    chain = prompt | llm | output_parser

    ans = chain.invoke({"question": result})

    return ans

    
@tool
def portfolio_allocation(query : str):
    """
    Based on the allocation stratergy provided as a query, change the allocation in the
    client stock portfolio in the SQL database
    """
    #query = Reallocte the portfolio for client 1 with a balanced strategy

    #  1. Read in the client database using the query database tool
    s1_query = f"Return the values of the stock holdings for client_id {query} in the sql database"
    query_database(s1_query)
    
    #  2. Determine allocation stratergy based on query
    
    

    #  3. Read in the Financials from the 10k

    #  4. Get current information from online

    #  5. Apply the stratergy we determined on the information from steps 3 and 4!

    #  6. Translate this into a percentage allocation for each company and write to database



    

tools = [Tool(name="search_tool", func=search_tool, description="Tool for performing online search operations"),
         Tool(name="create_embeddings", func=create_embeddings, description="Uses the content from the file path to create embeddings on the inputted data. Using the query provided, perform a similarity search on the query and return the output. If you can not provide a response,say I don't know rather than providing an incorrect response."),
        Tool(name='query_database', func=query_database, description= "This function queries the SQL database based on the query that the user provides to provide insight into the current client stock portfolio")]

In [151]:
template = """
You need to use the tools to find the appropriate action to take. 
Only use the query_database and portfolio_allocation tools when the term "portfolio" is mentioned in the query.
If further queries are presented alongside the term "portfilio", then use all tools available.
Otherwise, ignore these two tools and use the embeddings first
to see if you can get relevant responses from the similarity search from the user's query. If you do get relevant
responses, please return the responses from the similarity search. If you can't get relevent results from the embeddings,
use the seatch tool and get the reponse from there. If you still can't find an answer from an online search, please do not make up an
answer and return 'Sorry I can not answer your query' instead.

You have access to the following tools:
{tools}

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""

# Define the query and create the prompt
#query = 'return the values for client_id 1 in the sql database and also tell me the current apple stock price'
#query = 'Tell me the current apple stock price'
query = 'Return me the stock allocation for client 1 and client 2'
#query = "The Company continues to develop new technologies to enhance existing products and services and to expand the range of its offerings through research and development RD licensing of intellectual property and acquisition of thirdparty businesses and technology"
#query = "Give me a sentence from the apple 10-k report"
#query = "what is the net sales of Macs in 2021"
#query =  "add a new client to the database with random stock allocations, provide the sql query to do so and execute it"
#query =  "set the stock allocations for client_id 1 to 5,5,90. provide the sql query to do so and execute it on the database" #remeber to pull numbers from another query into this framework

prompt = PromptTemplate(input_variables=['input','tools', 'agent_scratchpad', 'tool_names'], template=template)

# Initialize the language model
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)


# Create the agent using the language model and the toolset
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Execute the agent with the given input
response = agent_executor.invoke({'input': query, 'tools' : tools, 'tool_names' : [tool.name for tool in tools]})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the query_database tool to retrieve the stock allocation for client 1 and client 2.

Action: query_database

Action Input: query_database(query="stock allocation for client 1 and client 2")


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


portfolio
Invoking: `sql_db_schema` with `{'table_names': 'portfolio'}`



CREATE TABLE portfolio (
	"ClientId" INTEGER, 
	"AAPL" DECIMAL(2, 2) NOT NULL, 
	"MSFT" DECIMAL(2, 2), 
	"NVIDIA" DECIMAL(2, 2), 
	PRIMARY KEY ("ClientId")
)

/*
3 rows from portfolio table:
ClientId	AAPL	MSFT	NVIDIA
1	0.40	0.20	0.40
2	0.25	0.25	0.50
3	0.25	0.25	0.50
*/
Invoking: `sql_db_query` with `{'query': 'SELECT * FROM portfolio WHERE ClientId IN (1, 2)'}`


[(1, 0.4, 0.2, 0.4), (2, 0.25, 0.25, 0.5)]The stock allocation for Client 1 is as follows:
- AAPL: 40%
- MSFT: 20%
- NVIDIA: 40%

The stock allocation for Client 2 is as follows:
- AAPL: 25%
- MSFT: 25%
- NVIDIA: 50%

> Finish

KeyError: 'template'

In [174]:
query = 'Return me the stock allocation fro client 1 and client 2'
query_database(query)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


portfolio
Invoking: `sql_db_schema` with `{'table_names': 'portfolio'}`



CREATE TABLE portfolio (
	"ClientId" INTEGER, 
	"AAPL" DECIMAL(2, 2) NOT NULL, 
	"MSFT" DECIMAL(2, 2), 
	"NVIDIA" DECIMAL(2, 2), 
	PRIMARY KEY ("ClientId")
)

/*
3 rows from portfolio table:
ClientId	AAPL	MSFT	NVIDIA
1	0.40	0.20	0.40
2	0.25	0.25	0.50
3	0.25	0.25	0.50
*/
Invoking: `sql_db_query` with `{'query': 'SELECT * FROM portfolio WHERE ClientId IN (1, 2)'}`


[(1, 0.4, 0.2, 0.4), (2, 0.25, 0.25, 0.5)]The stock allocation for client 1 is 40% AAPL, 20% MSFT, and 40% NVIDIA.
The stock allocation for client 2 is 25% AAPL, 25% MSFT, and 50% NVIDIA.

> Finished chain.


OutputParserException: Got invalid return object. Expected key `client` to be present, but got {'client 1': 'The stock allocation for client 1 is 40% AAPL, 20% MSFT, and 40% NVIDIA.', 'client 2': 'The stock allocation for client 2 is 25% AAPL, 25% MSFT, and 50% NVIDIA.'}